In [2]:
import numpy as np
import pandas as pd
import sys
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.optimize import curve_fit
import matplotlib.ticker as ticker
import math
import plotly.graph_objects as go
sys.path.append('..\\custom_libraries')

# Custom library to analyze NC rate of deterioration
import rate_analysis
df = pd.read_csv('../../spreadsheet_data/i16WB_tag_replaced.csv')

In [3]:
def plot_line_chart(df, buffer_start, buffer_end, bucket_size, max_state=3):
    x_val = []
    groups = math.ceil(max_state / bucket_size) + 1
    fig, ax = plt.subplots()
    for i in range(groups):
        x_val.append('[' + str(i * bucket_size) + ', ' + str((i+1) * bucket_size) + ')')
    for buffer in range(buffer_start, buffer_end + 1):
        chance = []
        det_arr = np.array([])
        avg_arr = np.array([])
        initial_state_arr = np.array([])   
        for year in range(2014, 2018):
            det = rate_analysis.has_deteriorated(df, str(year), str(year+1))
            avg = rate_analysis.calc_average_state_by_slab(df, str(year), buffer)
            inital_state = df[str(year)].to_numpy()
            det_arr = np.append(det_arr, det)
            avg_arr = np.append(avg_arr, avg)
            initial_state_arr = np.append(initial_state_arr, inital_state)
            
        

        state_df = pd.DataFrame({'det': det_arr, 'avg' : avg_arr, 'initial_state': initial_state_arr})

        state_df = state_df[state_df['initial_state'] != 'SS']
        state_df = state_df[state_df['initial_state'] != 'R']
        state_df = state_df[state_df['initial_state'] != 'B']

        for i in range(0, groups):
            grouping = state_df[(state_df['avg'] >= i*bucket_size) & (state_df['avg'] < (i+1)*bucket_size)]
            if len(grouping) == 0:
                chance.append(0)
            else:
                chance.append(len(grouping[grouping['det'] == True])/len(grouping))
        ax.plot(x_val, chance, label=str(buffer))
    
    ax.set_xlabel('Average State')
    ax.set_ylabel('Chance of Deterioration')
    ax.set_title('Rate of Deterioration Based on Average State for Multiple Buffers')
    ax.legend(title='Buffer Size', loc='upper left')
    fig.set_size_inches(20, 7)

    
    


In [4]:
def plot_line_chart(df, buffer_start, buffer_end, bucket_size, max_state=3):
    x_val = []
    groups = math.ceil(max_state / bucket_size) + 1
    fig = go.Figure()
    for i in range(groups):
        x_val.append('[' + str(i * bucket_size) + ', ' + str((i+1) * bucket_size) + ')')
    for buffer in range(buffer_start, buffer_end + 1):
        chance = []
        det_arr = np.array([])
        avg_arr = np.array([])
        initial_state_arr = np.array([])   
        for year in range(2014, 2018):
            det = rate_analysis.has_deteriorated(df, str(year), str(year+1))
            avg = rate_analysis.calc_average_state_by_slab(df, str(year), buffer)
            inital_state = df[str(year)].to_numpy()
            det_arr = np.append(det_arr, det)
            avg_arr = np.append(avg_arr, avg)
            initial_state_arr = np.append(initial_state_arr, inital_state)
            
        

        state_df = pd.DataFrame({'det': det_arr, 'avg' : avg_arr, 'initial_state': initial_state_arr})

        state_df = state_df[state_df['initial_state'] != 'SS']
        state_df = state_df[state_df['initial_state'] != 'R']
        state_df = state_df[state_df['initial_state'] != 'B']

        for i in range(0, groups):
            grouping = state_df[(state_df['avg'] >= i*bucket_size) & (state_df['avg'] < (i+1)*bucket_size)]
            if len(grouping) == 0:
                chance.append(0)
            else:
                chance.append(len(grouping[grouping['det'] == True])/len(grouping))
        fig.add_trace(go.Scatter(x=x_val, y=chance, mode='lines', name=str(buffer)))
    
    # Update layout
    fig.update_layout(
    title='Sensitivity Analysis: How Average Pavement State Influences Deterioration Rate',
    xaxis=dict(title='Average Pavement State'),
    yaxis=dict(title='Chance of Deterioration'),
    showlegend=True  # Show legend
    )

    # Show the plot
    fig.show()
                      
    


        
    
  

# A Look At Slab Buffers 1-20 for All Slabs
Overall, variation increases when average pavement state is 2 or greater among the buffers. This is because at higher slab buffers, it is harder to achieve a higher average pavement state value. At slab buffers 18, 19, and 20, chance of deterioration is 0 at average pavement state [2, 2.5) indicating a small (or no) sample size for slabs that register that average pavement state at that certain buffer value. A higher sample size might help smooth out the variation for higher slab buffers and ultimately help produce stronger conclusions as to what slab buffers to use.

In [15]:
plot_line_chart(df, 1, 20, .5, 3)

In [9]:
plot_line_chart(df, 1, 10, .5, 3)

In [7]:
plot_line_chart(df, 1, 5, .5, 3)

In [8]:
plot_line_chart(df, 1, 3, .5, 3)

In [13]:
plot_line_chart(df, 150, 150, .5, 3)